### 添加未登录词(字)

In [4]:
from transformers import BertTokenizer
import torch
token = BertTokenizer.from_pretrained("./")
sentence = "中核釱白怎么样？"
sentence = "焦赟这个人怎么样？"
token.convert_tokens_to_ids(token.tokenize(sentence))

[4193, 1, 6821, 702, 782, 2582, 720, 3416, 8043]

### 未登录词(字)太多了，超过unused的个数   
#### 评估一下未登录词(字)对任务的影响，如果影响很小的话，unused解决一部分，剩下的可以忽略不计；  
#### 如果忽略这些未登录词(字)确实有很大影响的话，那就添加字典吧,并且同时修改权重参数尺寸，修改模型尺寸

In [7]:
from transformers import BertTokenizer,BertForMaskedLM,BertConfig,BertModel
token = BertTokenizer.from_pretrained("./")
sentence = "焦赟这个人怎么样？"
token.convert_tokens_to_ids(token.tokenize(sentence))


[4193, 21128, 6821, 702, 782, 2582, 720, 3416, 8043]

In [8]:
config = BertConfig.from_json_file("./config.json")
model = BertForMaskedLM(config=config)

In [20]:
model_weight = torch.load("./pytorch_model.bin")
# for item in ["bert.embeddings.word_embeddings.weight","cls.predictions.bias","cls.predictions.decoder.weight"]:
#     del model_weight[item]

for item in ["bert.embeddings.word_embeddings.weight","cls.predictions.bias","cls.predictions.decoder.weight"]:
    model_weight[item] = torch.cat([model_weight[item],model_weight[item][0:1]],dim=0)

In [18]:
# model_weight["bert.embeddings.word_embeddings.weight"].shape

torch.Size([21128, 768])

In [21]:
model.load_state_dict(model_weight,strict=False)

_IncompatibleKeys(missing_keys=['bert.embeddings.position_ids', 'cls.predictions.decoder.bias'], unexpected_keys=['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias'])

In [19]:
model.get_parameter("bert.embeddings.word_embeddings.weight").shape

torch.Size([21129, 768])

### 添加未登录词(词)

In [22]:
from transformers import BertTokenizer
import jieba
import torch
token = BertTokenizer.from_pretrained("./")
sentence = "万里长城是世界七大奇迹之一"
token.tokenize(sentence)

['万', '里', '长', '城', '是', '世', '界', '七', '大', '奇', '迹', '之', '一']

In [27]:
jieba.add_word("万里长城")
jieba.lcut(sentence)
ls = jieba.lcut(sentence)
ls = ls[0:1]+list("".join(ls[1:]))
ls

['万里长城', '是', '世', '界', '七', '大', '奇', '迹', '之', '一']

In [28]:
token.convert_tokens_to_ids(ls)

[100, 3221, 686, 4518, 673, 1920, 1936, 6839, 722, 671]

### wwm(whole word mask)

In [ ]:
raw_text           使用语言模型来预测下一个词的概率。
CWS                使用 语言 模型 来 预测 下 一个 词 的 概率
Original Masking   使 用 语 言 [M] 型 来 [M] 测 下 一 个 词 的 概 率
WWM                使 用 语 言 [M] [M] 来 [M] [M] 下 一 个 词 的 概 率

In [31]:
#WWM
import numpy as np
sentence = "使用语言模型来预测下一个词的概率"
sentence = jieba.lcut(sentence)
mask_ids = np.random.choice(range(len(sentence)),2)
wwm = []
for i in range(len(sentence)):
    if i in mask_ids:
        wwm+=["[MASK]"]*len(sentence[i])
    else:
        wwm+=list(sentence[i])
wwm

['使',
 '用',
 '语',
 '言',
 '[MASK]',
 '[MASK]',
 '来',
 '预',
 '测',
 '下',
 '[MASK]',
 '[MASK]',
 '词',
 '的',
 '概',
 '率']

[103,
 103,
 6427,
 6241,
 3563,
 1798,
 3341,
 7564,
 3844,
 678,
 103,
 103,
 6404,
 4638,
 3519,
 4372]